In [0]:
%md

1. quick check app distinct between db and unified
2. download,revenue sum between db and unified


In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select count(distinct(app_id)) from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_0
    where 
        date = '2020-01-01'
    UNION ALL
    select distinct(app_id)
    from aa.app_store_daily_estimate_1
    where 
        date = '2020-01-01'
    UNION ALL
    select distinct(app_id)
    from aa.app_store_daily_estimate_2
    where 
        date = '2020-01-01'

\$proxy\$) tbl (app_id BIGINT);

EOF


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 

select distinct(device_id) from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct(device_id)
    from aa.app_store_daily_estimate_2
    where 
        date = '2020-01-01'
\$proxy\$) tbl (device_id SMALLINT);

EOF


In [0]:


import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_0
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
""".format(date)

    db_df = query_df(daily_est_dsn, sql)
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/device_code=android-all/".format(date)).select("app_id").distinct().toPandas()

    u_list = unified_df.app_id.unique()
    d_list = db_df.app_id.unique()
    diff_list =  [x for x in  d_list if x not in u_list ]
     
    if len(diff_list) ==  0:
        print "{} PASS".format(date)
    else:
        print "{} FAIL".format(date)
        print "{} {} {}".format(len(u_list), len(d_list), diff_list[0:5])
        # print len(u_list)
        # print len(d_list)
        # # print [x for x in  u_list if x not in d_list ]
        # print [x for x in  d_list if x not in u_list ]
        # print '*' * 200
        


date_list = get_date_list("2019-07-01", "2020-02-21")
for date in date_list:
    # compare(date)
    try:
        compare(date)
    except Exception, e:
        print "{}: ERROR".format(date)
        print e.message



In [0]:

print 123

In [0]:

import boto3

some_binary_data = b'Here we have some data'
more_binary_data = b'Here we have some more data'

# # Method 1: Object.put()
# s3 = boto3.resource('s3')
# object = s3.Object('s3://b2c-prod-data-pipeline-qa/', 'tom/r')
# object.put(Body=some_binary_data)

# # Method 2: Client.put_object()
# client = boto3.client('s3')
# client.read_object(Body=more_binary_data, Bucket='b2c-prod-data-pipeline-qa', Key='my/key/including/anotherfilename.txt')

def read_s3(key):
    print '*'*100 
    print key
    s3 = boto3.resource('s3')
    obj = s3.Object('b2c-prod-data-pipeline-qa', key)
    body = obj.get()['Body'].read()
    print body
    

read_s3('tom/regression1.txt')
read_s3('tom/regression2.txt')
read_s3('tom/regression3.txt')
read_s3('tom/regression4.txt')
read_s3('tom/regression5.txt')



In [0]:

import boto3

def read_s3(key):
    print '*'*100 
    print key
    s3 = boto3.resource('s3')
    obj = s3.Object('b2c-prod-data-pipeline-qa', key)
    body = obj.get()['Body'].read()
    print body

read_s3('tom/all3/regression1.txt')


In [0]:
%%sh


# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v2/fact/granularity=daily/date=2020-01-01/
aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/all/
# aws s3 ls s3://prod.appannie.aso/search_volume/

# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-02-20/







In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
set search_path=aso;
-- select * from aso.aso_keyword_trafficshare_fact_v1 order by date desc limit 3;
-- select app_id_list,traffic_share_list from aso.aso_keyword_trafficshare_fact_v1 where date='2020-02-15' and device_code='ios-phone' limit 3
-- select * from aso.aso_keyword_trafficshare_fact_v1 where keyword_id='6297851' AND date BETWEEN '2019-12-01' AND '2020-02-22' limit 3;
select * from aso.aso_sv_kd_fact_v1  where keyword_id=6297851 AND date BETWEEN '2020-01-22' AND '2020-02-22' limit 3;
select * from aso.aso_sv_kd_fact_v1  where keyword_id=6297851 AND date BETWEEN '2019-12-22' AND '2020-01-22' limit 3;
select * from aso.aso_sv_kd_fact_v1  where keyword_id=6297851 AND date BETWEEN '2019-11-22' AND '2019-12-22' limit 3;
\d+ aso.aso_sv_kd_fact_v1



EOF


In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
set search_path=aso;
-- select * from aso.aso_keyword_trafficshare_fact_v1 order by date desc limit 3;
-- select app_id_list,traffic_share_list from aso.aso_keyword_trafficshare_fact_v1 where date='2020-02-15' and device_code='ios-phone' limit 3
select * from aso.aso_sv_kd_fact_v1  where keyword_id='6297851' AND date BETWEEN '2020-01-22' AND '2020-02-22' limit 3;
select * from aso.aso_sv_kd_fact_v1  where keyword_id='6297851' AND date BETWEEN '2019-12-22' AND '2020-01-22' limit 3;
select * from aso.aso_sv_kd_fact_v1  where keyword_id='6297851' AND date BETWEEN '2019-11-22' AND '2019-12-22' limit 3;


EOF


In [0]:
%%sh

aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-01-22/


In [0]:



spark.read.parquet("s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2020-01-22/").show(1)





In [0]:
%%sh


# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v4/fact/granularity=daily/date=2020-09-15/data_stage=preview/device_code=ios-all/

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-09-15/device_code=ios-phone/



store.store_est_fact_v2

In [0]:


spark.read.format('delta').load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-dna-log.v1/fact/granularity=daily/date=2020-09-15/device_code=ios-phone/").filter("app_id=284882215 AND country_code='US' ").show(10)


In [0]:
%%sh
